In [20]:
from lib.knn.__init__ import KNearestNeighbor
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
from lib.network import ConfNet, PoseNetRGBOnly
from lib.loss import Loss, LossConf, LossNoConf
from lib.transformations import euler_matrix, quaternion_matrix, quaternion_from_matrix
from datasets.linemod.dataset import PoseDataset as PoseDataset_linemod
import yaml
import numpy as np
import warnings
import os
import time
warnings.filterwarnings("ignore")

In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
num_objects = 5
num_points = 500
outf = 'trained_models/linemod'
log_dir = 'experiments/logs/linemod'
repeat_epoch = 20
decay_margin = 0.016
decay_start = False
lr_rate = 0.3
w_rate = 0.3

In [18]:
# model = 'trained_models/linemod/rgb_model_4_0.010808611272843284.pth'
posenet = PoseNetRGBOnly(num_points = num_points, num_obj = num_objects)
posenet.cuda();
# posenet.load_state_dict(torch.load(model))
# for param in posenet.parameters():
#     param.requires_grad=False
# posenet.eval();

In [5]:
confnet = ConfNet(num_objects)
confnet.cuda();

In [10]:
dataset_root = "./datasets/linemod/Linemod_preprocessed"
noise_trans = 0.03
refine_start = False
decay_start = False


dataset = PoseDataset_linemod('train', 
                              num_points, 
                              True, 
                              dataset_root, 
                              noise_trans, 
                              refine_start, use_true_depth=True)

workers = 4
dataloader = torch.utils.data.DataLoader(dataset, 
                                         batch_size=1, 
                                         shuffle=True, 
                                         num_workers=workers)

test_dataset = PoseDataset_linemod('test', 
                                   num_points, 
                                   False, 
                                   dataset_root, 
                                   0.0, 
                                   refine_start, use_true_depth=True)

testdataloader = torch.utils.data.DataLoader(test_dataset, 
                                             batch_size=1, 
                                             shuffle=False, 
                                             num_workers=workers)

sym_list = dataset.get_sym_list()
num_points_mesh = dataset.get_num_points_mesh()

Object 2 buffer loaded
Object 4 buffer loaded
Object 5 buffer loaded
Object 10 buffer loaded
Object 11 buffer loaded
Object 2 buffer loaded
Object 4 buffer loaded
Object 5 buffer loaded
Object 10 buffer loaded
Object 11 buffer loaded


In [21]:
criterion_conf = LossConf(num_points_mesh, sym_list)
criterion = LossNoConf(num_points_mesh, sym_list)

In [22]:
start_epoch = 1
nepoch = 16
w = 0.015
batch_size = 8

In [23]:
lr = 0.0001
optimizer = optim.Adam(posenet.parameters(), lr=lr)

In [27]:
best_test = np.Inf
print_every = 50
st_time = time.time()

for epoch in range(start_epoch, nepoch):
    mess = 'Train time {0}'.format(time.strftime("%Hh %Mm %Ss", time.gmtime(time.time() - st_time)) + ', ' + 'Training started')
    print(mess)
    
    train_count = 0
    train_dis_conf_avg = 0.0
    train_dis_avg = 0.0
    
#     confnet.train()
    posenet.train()
    optimizer.zero_grad()

    for rep in range(repeat_epoch):
        for i, data in enumerate(dataloader, 0):
            points, choose, img, target, model_points, idx = data 
            points, choose, img, target, model_points, idx = Variable(points).cuda(), \
                                                             Variable(choose).cuda(), \
                                                             Variable(img).cuda(), \
                                                             Variable(target).cuda(), \
                                                             Variable(model_points).cuda(), \
                                                             Variable(idx).cuda()
            
            pred_r, pred_t, pred_c, emb = posenet(img, choose, idx)
            
#             pred_loss = confnet(emb)
            
#             loss_conf, dis_conf = criterion_conf(pred_loss, pred_r, pred_t, pred_c, target, 
#                                   model_points, idx,
#                                   points, w, refine_start)
            loss = criterion(pred_r, pred_t, pred_c, target, 
                                  model_points, idx,
                                  points, w, refine_start)
      
#             loss_conf.backward()
            loss.backward()
            
#             train_dis_avg += dis.item()
            train_dis_avg += loss.item()
#             train_dis_conf_avg += dis_conf.item()
            train_count += 1
            if train_count % batch_size == 0:
                
                mess = 'Train time {0} Epoch {1} Batch {2} Frame {3} Avg_Loss:{4} Conf_loss: {5}'.format(
                        time.strftime("%Hh %Mm %Ss", time.gmtime(time.time() - st_time)), 
                        epoch, int(train_count / batch_size), 
                        train_count, 
                        train_dis_avg / batch_size, 
                        train_dis_conf_avg / batch_size)
                
                
                optimizer.step()
                optimizer.zero_grad()
                
                train_dis_avg = 0
                train_dis_conf_avg = 0
                
                
                if train_count % print_every == 0:
                    print(mess)
            
            if train_count != 0 and train_count % 500 == 0:                    
#                 torch.save(confnet.state_dict(), '{0}/conf_model_current.pth'.format(outf))
                torch.save(posenet.state_dict(), '{0}/pose_model_current.pth'.format(outf))
            
    print('>>>>>>>>----------epoch {0} train finish---------<<<<<<<<'.format(epoch))
    
    test_dis = 0.0
    test_conf_dis = 0.0
    test_count = 0
#     confnet.eval()
    posenet.eval()
    
    for j, data in enumerate(testdataloader, 0):
        points, choose, img, target, model_points, idx = data
        points, choose, img, target, model_points, idx = Variable(points).cuda(), \
                                                         Variable(choose).cuda(), \
                                                         Variable(img).cuda(), \
                                                         Variable(target).cuda(), \
                                                         Variable(model_points).cuda(), \
                                                         Variable(idx).cuda()
       
        pred_r, pred_t, pred_c, emb = posenet(img, choose, idx)     
#         pred_loss = confnet(emb)
            
#         loss_conf, dis_conf = criterion_conf(pred_loss, pred_r, pred_t, pred_c, target, 
#                               model_points, idx,
#                               points, w, refine_start)
        
#         loss, dis, _, _ = criterion(pred_r, pred_t, pred_c, target, 
#                               model_points, idx,
#                               points, w, refine_start)
        loss = criterion(pred_r, pred_t, pred_c, target, 
                                  model_points, idx,
                                  points, w, refine_start)
#         test_dis += dis.item()
        test_dis += loss.item()
#         test_conf_dis += dis_conf.item()

        test_count += 1
    test_dis = test_dis / test_count
    test_conf_dis = test_conf_dis / test_count
    mess = 'Test time {0} Epoch {1} TEST FINISH Avg dis: {2} Avg conf loss: {3}'.format(time.strftime("%Hh %Mm %Ss", 
                                                                                   time.gmtime(time.time() - st_time)), 
                                                                                    epoch, test_dis, test_conf_dis)
    print(mess)
    
    if test_conf_dis <= best_test: 
        best_test = test_conf_dis
#         torch.save(confnet.state_dict(), '{0}/conf_model_{1}_{2}.pth'.format(outf, epoch, test_conf_dis))
        torch.save(posenet.state_dict(), '{0}/pose_noconf_model_{1}_{2}.pth'.format(outf, epoch, test_conf_dis))
        print(epoch, '>>>>>>>>----------BEST TEST MODEL SAVED---------<<<<<<<<')
        

Train time 00h 00m 00s, Training started
Train time 00h 00m 10s Epoch 1 Batch 25 Frame 200 Avg_Loss:0.0773667513858527 Conf_loss: 0.0
Train time 00h 00m 21s Epoch 1 Batch 50 Frame 400 Avg_Loss:0.05309214978478849 Conf_loss: 0.0
Train time 00h 00m 31s Epoch 1 Batch 75 Frame 600 Avg_Loss:0.04698517231736332 Conf_loss: 0.0
Train time 00h 00m 42s Epoch 1 Batch 100 Frame 800 Avg_Loss:0.06604087783489376 Conf_loss: 0.0
Train time 00h 00m 52s Epoch 1 Batch 125 Frame 1000 Avg_Loss:0.06950244668405503 Conf_loss: 0.0
Train time 00h 01m 03s Epoch 1 Batch 150 Frame 1200 Avg_Loss:0.06278516235761344 Conf_loss: 0.0
Train time 00h 01m 13s Epoch 1 Batch 175 Frame 1400 Avg_Loss:0.056021313881501555 Conf_loss: 0.0
Train time 00h 01m 24s Epoch 1 Batch 200 Frame 1600 Avg_Loss:0.07542966189794242 Conf_loss: 0.0
Train time 00h 01m 34s Epoch 1 Batch 225 Frame 1800 Avg_Loss:0.04698366788215935 Conf_loss: 0.0
Train time 00h 01m 44s Epoch 1 Batch 250 Frame 2000 Avg_Loss:0.04393592069391161 Conf_loss: 0.0
Train 

KeyboardInterrupt: 